In [1]:
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
# bench = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)

In [2]:
import json

data = []
with open('final_result/RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl', 'r', encoding='utf-8') as file_an:
        for line in file_an:
            data.append(json.loads(line)) 

tmp = {}
new_data = []
count = 0
for i in range(0, 400, 4):
    if i + 4 > len(data):
        break
    while data[i]['book_title'] != bench.collection[count]['title']:
            #print(data[i]['book_title'])
            #print(collection[count]['title'])
        count += 1
    tmp = {
        'title': data[i]['book_title'], 
        'author': bench.collection[count]['author'],
        'gold_annotation': bench.collection[count]['annotation'],
        'annotations': {
            'blueprint': data[i]['annotation'],
            'blueprint_cluster': data[i + 1]['annotation'],
            'hierarchical': data[i + 2]['annotation'],
            'hierarchical_filtered': data[i + 3]['annotation']
        }
    }
    new_data.append(tmp)
    count += 1

await bench.evaluater.get_all_data_qa(new_data)

In [2]:
bench = Summarisation(URL=url, KEY=key, model_name='tpro', device=device, encoder=encoder)

In [3]:
print(await bench.client.get_completion('hi', max_tokens=8))

ChatCompletion(id='chatcmpl-902250d6b6ea4f48a2fcab7f9bbe2c37', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1754154395, model='/workdir/models/T-pro-it-1.0', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=8, prompt_tokens=9, total_tokens=17, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)


In [22]:
lst = list(await bench.client.client.models.list())[0][1]
for l in lst:
    print(l)

Model(id='ruadapt-qwen3-4b', created=None, object=None, owned_by=None, max_model_len=40960, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='tpro', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='yagpt5lite', created=None, object=None, owned_by=None, max_model_len=30000, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-32B-Pro-Beta', created=None, object=None, owned_by=None, max_model_len=28000, status='spawned')
Model(id='RefalMachine/RuadaptQwen3-32B-Instruct-v2', created=None, object=None, owned_by=None, max_model_len=24000, status='spawned')
Model(id='llama3-70b', created=None, object=None, owned_by=None, max_model_len=14000, status='spawned')
Model(id='Qwen3-235B-A22B-Instruct-2507', created=None, object=None, owned_by=None, max_model_len=-1, status='spawned')
Model(id='RefalMachine/RuadaptQwen2.5-7B-Garant-v2', created=

In [2]:
from utils import chunk_text
import asyncio
import json
from datetime import datetime, timezone
import aiofiles
import os
import time

async def run_bench_for_model(
    url: str,
    key: str,
    model_name: str,
    device = None,
    encoder = None,
    out_dir: str = "cluster_new",
    total = 0, # CHANGE THAT
    start_point = 0 # AND THAT TOO
):

    file_path = f"{out_dir}/{model_name.replace('/', '_')}.jsonl"
    os.makedirs(out_dir, exist_ok=True)
    bench = Summarisation(URL=url, KEY=key, model_name=model_name, device=device, encoder=encoder)
    total_count = total
    async with aiofiles.open(file_path, "a", encoding="utf‑8") as f:
        for item in bench.collection[start_point:]:
            if total_count == 100:
                print('DONE')
                break
            text = '\n'.join(item['text'])
            if len(text) > 800000:
                continue
            chunks = chunk_text(text)
            #s1 = await bench.blueprint.run(chunks, initial_word_limit=500, mode='default')
            #await f.write(_as_jsonl(model_name, item['title'], item['author'], "blueprint", s1))
            
            s2 = await bench.blueprint.run(chunks, initial_word_limit=500, mode='cluster')
            await f.write(_as_jsonl(model_name, item['title'], item['author'], "blueprint_cluster", s2))
            
            #s3 = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=False)
            #await f.write(_as_jsonl(model_name, item['title'], item['author'], "hierarchical", s3))
            
            #s4 = await bench.hierarchical.run(chunks, initial_word_limit=500, filtered=True)
            #await f.write(_as_jsonl(model_name, item['title'], item['author'], "hierarchical_filtered", s4))
            
            total_count += 1

def _as_jsonl(model, book_title, author, method, annotation):
    entry = {
        "ts": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "model": model,
        "book_title": book_title,
        "author": author,
        "method": method,
        "annotation": annotation
    }
    return json.dumps(entry, ensure_ascii=False) + "\n"

In [3]:
from utils import AsyncList

async def main():
    models = AsyncList()
    model_names = [
        'RefalMachine/RuadaptQwen3-32B-Instruct-v2',
        'Qwen3-235B-A22B-Instruct-2507',
        'DeepSeek V3',
        'tpro',
        'RefalMachine/RuadaptQwen2.5-7B-Lite-Beta',
        'yagpt5lite'
    ]
    for name in model_names:
        models.append(run_bench_for_model(url, key, name, device, encoder))

    await models.complete_couroutines(batch_size=len(model_names))
    models = await models.to_list()

In [4]:
await main()

APITimeoutError: Request timed out.

DONE
DONE


In [4]:
bench = Summarisation(URL=url, KEY=key, model_name='yagpt5lite', device=device, encoder=encoder)

In [14]:
#bench = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)
print(len(''.join(bench.collection[101]['text'])))

8178


In [8]:
print(len(data))

400


In [257]:
from datetime import datetime, timedelta
import itertools
from collections import defaultdict
import json
import os
import numpy as np

for fname in sorted(os.listdir('final_result')):
    if not fname.endswith('.jsonl'):
        continue
    path = os.path.join('final_result', fname)
    data = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
    
    timestamps = [e['ts'] for e in data]
    datetimes = [datetime.fromisoformat(ts) for ts in timestamps]
    
    category_cycle = itertools.cycle([1, 2, 3, 4])
    category_seconds = defaultdict(list)
    
    first_diff = timedelta(minutes=2)
    first_cat = next(category_cycle)
    category_seconds[first_cat].append(first_diff.total_seconds())
    
    for prev, curr in zip(datetimes, datetimes[1:]):
        diff_sec = (curr - prev).total_seconds()
        cat = next(category_cycle)
        if diff_sec > 4000:
            continue
        category_seconds[cat].append(diff_sec)
    
    print(fname)
    for cat in sorted(category_seconds):
        diffs = np.array(category_seconds[cat], dtype=float)
        mean = np.mean(diffs) if diffs.size else float('nan')
        std = np.std(diffs) if diffs.size else float('nan')
        print(f"{mean:.2f} ± {std:.2f} & ", end='')
    print()
    print()


DeepSeek V3.jsonl
315.67 ± 321.89 & 132.60 ± 197.25 & 196.77 ± 187.85 & 147.21 ± 146.41 & 

Qwen3-235B-A22B .jsonl
230.35 ± 271.03 & 158.30 ± 196.35 & 103.49 ± 97.30 & 83.06 ± 102.05 & 

RefalMachine_RuadaptQwen2.5-7B-Lite-Beta.jsonl
126.84 ± 145.74 & 76.66 ± 91.78 & 68.86 ± 64.85 & 53.59 ± 47.28 & 

RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl
379.24 ± 500.40 & 286.35 ± 395.97 & 218.30 ± 195.16 & 166.79 ± 164.61 & 

tpro.jsonl
592.39 ± 772.19 & 247.59 ± 361.20 & 367.32 ± 324.49 & 267.73 ± 253.34 & 

yagpt5lite.jsonl
113.34 ± 123.78 & 42.15 ± 56.50 & 31.02 ± 28.51 & 27.39 ± 28.05 & 



In [6]:
import json

data_qa = []
with open('qa_data/data.jsonl', 'r') as file:
    for line in file:
        data_qa.append(json.loads(line))

In [8]:
tmp = {}
new_data = []
count = 0
for i in range(0, 400, 4):
    if i + 4 > len(data):
        break
    while data[i]['book_title'] != bench.collection[count]['title']:
            #print(data[i]['book_title'])
            #print(collection[count]['title'])
        count += 1
    tmp = {
        'title': data[i]['book_title'], 
        'author': bench.collection[count]['author'],
        'gold_annotation': bench.collection[count]['annotation'],
        'questions': data_qa[int(i / 4)]['questions'],
        'answers': data_qa[int(i / 4)]['answers'],
        'annotations': {
            'blueprint': data[i]['annotation'],
            'blueprint_cluster': data[i + 1]['annotation'],
            'hierarchical': data[i + 2]['annotation'],
            'hierarchical_filtered': data[i + 3]['annotation']
        }
    }
    new_data.append(tmp)
    count += 1


In [9]:
print(len(new_data))

100


In [ ]:
# CALCULATING QA METRIC FOR MODELS

import time
import asyncio
import json
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(URL=url, KEY=key, model_name='yagpt5lite', device=device, encoder=encoder)

files = [
    'final_result/DeepSeek V3.jsonl',
    'final_result/Qwen3-235B-A22B .jsonl',
    'final_result/RefalMachine_RuadaptQwen2.5-7B-Lite-Beta.jsonl',
    'final_result/RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl',
    'final_result/tpro.jsonl',
    'final_result/yagpt5lite.jsonl'
]

with open('combined_data.json', 'r', encoding='utf-8') as f:
    collection = json.load(f)

data_qa = []
with open('qa_data/data.jsonl', 'r') as file:
    for line in file:
        data_qa.append(json.loads(line))


for file_name in files:
    data = []
    with open(file_name, 'r') as file_texts:
        for line in file_texts:
            data.append(json.loads(line))

    tmp = {}
    new_data = []
    count = 0
    for i in range(0, 400, 4):
        if i + 4 > len(data):
            break
        while data[i]['book_title'] != collection[count]['title']:
                #print(data[i]['book_title'])
                #print(collection[count]['title'])
            count += 1
        tmp = {
            'title': data[i]['book_title'], 
            'author': collection[count]['author'],
            'gold_annotation': collection[count]['annotation'],
            'questions': data_qa[int(i / 4)]['questions'],
            'answers': data_qa[int(i / 4)]['answers'],
            'annotations': {
                'blueprint': data[i]['annotation'],
                'blueprint_cluster': data[i + 1]['annotation'],
                'hierarchical': data[i + 2]['annotation'],
                'hierarchical_filtered': data[i + 3]['annotation']
            }
        }
        new_data.append(tmp)
        count += 1
    
    error_file = 'qa_rerun/' + file_name.split('/')[-1][:-6] + '.txt'
    data_file = 'qa_rerun/' + file_name.split('/')[-1]
    with open(error_file, 'a', encoding='utf-8') as file, open(data_file, 'a', encoding='utf-8') as f_data:
        for item in new_data:
            try:
                #print(item['title'])
                for ann in item['annotations'].values():
                    c, s, a_g = await bench.evaluater.compute_similarity(item['gold_annotation'], ann, item['questions'], item['answers'])
                    #print(c, s)
                    entry = {
                        'title': item['title'],
                        'author': item['author'],
                        'questions': item['questions'],
                        'answers_gold': item['answers'],
                        'answers_gen': a_g,
                        'coverage': c,
                        'similarity': s
                    }
                    f_data.write(json.dumps(entry, ensure_ascii=False) + '\n')
            except (KeyboardInterrupt, asyncio.CancelledError):
                raise
            except:
                raise
                file.write(f"{item['title']} {item['author']}\n")
                time.sleep(120)
                continue

In [12]:
data_c = []
with open('answers_gen.jsonl', 'r') as file:
    for line in file:
        data_c.append(json.loads(line))

In [30]:
methods = [
    "blueprint",
    "blueprint_cluster",
    "hierarchical",
    "hierarchical_filtered",
]
row = []
for i in range(0, 400, 4):
    c = 0
    for m in methods:
        row.append({
            'model': 'yagpt5lite',
            'method': methods[c],
            'metric': 'coverage',
            'score': data_c[i + c]['coverage']
        })
        row.append({
            'model': 'yagpt5lite',
            'method': methods[c],
            'metric': 'similarity',
            'score': data_c[i + c]['similarity']
        })
        c += 1

In [3]:
import numpy as np

def mean_std(data):
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    return f"{100*mean:.2f} ± {100*std:.2f}"

methods = [
    "blueprint",
    "blueprint_cluster",
    "hierarchical",
    "hierarchical_filtered",
]

names = [
    'qa_rerun/DeepSeek V3.jsonl',
    'qa_rerun/Qwen3-235B-A22B .jsonl',
    'qa_rerun/RefalMachine_RuadaptQwen2.5-7B-Lite-Beta.jsonl',
    'qa_rerun/RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl',
    'qa_rerun/yagpt5lite.jsonl',
    'qa_rerun/tpro.jsonl'
]

print('QA coverage and similarity')
for name in names:
    data = []
    with open(name, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
        model_name = name.split('/')[1][:-6]
        row = []
        for i in range(0, 400, 4):
            c = 0
            for m in methods:
                row.append({
                    'model': model_name,
                    'method': methods[c],
                    'metric': 'coverage',
                    'score': data[i + c]['coverage']
                })
                row.append({
                    'model': model_name,
                    'method': methods[c],
                    'metric': 'similarity',
                    'score': data[i + c]['similarity']
                })
                c += 1
        row_c_b = [item['score'] for item in row if item['metric'] == 'coverage' and item['method'] == 'blueprint']
        row_s_b = [item['score'] for item in row if item['metric'] == 'similarity' and item['method'] == 'blueprint']
        row_c_bc = [item['score'] for item in row if item['metric'] == 'coverage' and item['method'] == 'blueprint_cluster']
        row_s_bc = [item['score'] for item in row if item['metric'] == 'similarity' and item['method'] == 'blueprint_cluster']
        row_c_h = [item['score'] for item in row if item['metric'] == 'coverage' and item['method'] == 'hierarchical']
        row_s_h = [item['score'] for item in row if item['metric'] == 'similarity' and item['method'] == 'hierarchical']
        row_c_hf = [item['score'] for item in row if item['metric'] == 'coverage' and item['method'] == 'hierarchical_filtered']
        row_s_hf = [item['score'] for item in row if item['metric'] == 'similarity' and item['method'] == 'hierarchical_filtered']
        print(model_name)
        print(f'{mean_std(row_c_h)} & {mean_std(row_c_b)} & {mean_std(row_c_hf)} & {mean_std(row_c_bc)}')
        print(f'{mean_std(row_s_h)} & {mean_std(row_s_b)} & {mean_std(row_s_hf)} & {mean_std(row_s_bc)}')
        print()

QA coverage and similarity
DeepSeek V3
53.46 ± 22.53 & 38.95 ± 23.26 & 43.76 ± 23.06 & 34.05 ± 22.36
42.36 ± 18.49 & 31.34 ± 18.99 & 34.81 ± 18.97 & 27.21 ± 18.09

Qwen3-235B-A22B 
50.03 ± 21.95 & 53.14 ± 20.63 & 43.95 ± 23.37 & 30.37 ± 21.35
39.61 ± 18.03 & 42.76 ± 17.17 & 35.55 ± 19.29 & 24.26 ± 17.30

RefalMachine_RuadaptQwen2.5-7B-Lite-Beta
18.91 ± 17.48 & 24.84 ± 21.21 & 19.97 ± 18.55 & 14.27 ± 14.11
14.66 ± 13.99 & 19.79 ± 17.26 & 15.68 ± 14.54 & 11.45 ± 11.89

RefalMachine_RuadaptQwen3-32B-Instruct-v2
31.46 ± 20.70 & 32.27 ± 22.33 & 32.30 ± 23.47 & 16.17 ± 15.35
24.29 ± 16.32 & 25.54 ± 17.86 & 25.00 ± 18.62 & 12.91 ± 12.58

yagpt5lite
36.73 ± 20.74 & 31.93 ± 21.62 & 32.31 ± 21.72 & 24.52 ± 17.98
29.78 ± 17.06 & 25.92 ± 18.31 & 26.27 ± 17.94 & 20.11 ± 15.06

tpro
38.41 ± 21.60 & 39.55 ± 21.96 & 35.36 ± 21.82 & 24.44 ± 17.44
30.59 ± 17.48 & 31.76 ± 18.21 & 28.28 ± 17.99 & 19.91 ± 14.92



In [31]:
print(row[1])

{'model': 'yagpt5lite', 'method': 'blueprint', 'metric': 'similarity', 'score': 0.5821428571428572}


In [10]:
with open('combined_data.json', 'r', encoding='utf-8') as f:
    collection = json.load(f)

data = []
with open('qa_results/DeepSeek V3.jsonl', 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line))
            
data_qa = []
with open('qa_data/data.jsonl', 'r', encoding='utf-8') as file_qa:
        for line in file_qa:
            data_qa.append(json.loads(line))

data_an = []
with open('final_result/RefalMachine_RuadaptQwen3-32B-Instruct-v2.jsonl', 'r', encoding='utf-8') as file_an:
        for line in file_an:
            data_an.append(json.loads(line))  
count = 0
new_data = []
for i in range(0, 400, 4):
    if i + 4 > len(data):
        break
    while data[i]['title'] != collection[count]['title']:
            #print(data[i]['book_title'])
            #print(collection[count]['title'])
        count += 1
    tmp = {
        'title': data[i]['title'], 
        'author': collection[count]['author'],
        'gold_annotation': collection[count]['annotation'],
        'questions': data_qa[int(i / 4)]['questions'],
        'answers': data_qa[int(i / 4)]['answers'],
        'annotations': {
            'blueprint': data_an[i]['annotation'],
            'blueprint_cluster': data_an[i + 1]['annotation'],
            'hierarchical': data_an[i + 2]['annotation'],
            'hierarchical_filtered': data_an[i + 3]['annotation']
        },
        'answers_gen': {
            'blueprint': [data[i]['answers_gen'], data[i]['coverage']],
            'blueprint_cluster': [data[i + 1]['answers_gen'], data[i + 1]['coverage']],
            'hierarchical': [data[i + 2]['answers_gen'], data[i + 2]['coverage']],
            'hierarchical_filtered': [data[i + 3]['answers_gen'], data[i + 3]['coverage']],
        }
    }
    new_data.append(tmp)
    count += 1

In [50]:
methods = [
    "blueprint",
    "blueprint_cluster",
    "hierarchical",
    "hierarchical_filtered",
]

row = []
for r1, b1 in zip(r, b):
    for r2, b2, m in zip(r1, b1, methods):
        row.append({
            'model': 'yagpt5lite',
            'method': m,
            'metric': 'rouge-l',
            'score': r2 * 100
        })
        row.append({
            'model': 'yagpt5lite',
            'method': m,
            'metric': 'bertscore',
            'score': b2[2] * 100
        })

In [ ]:
from utils import chunk_text
text = "\n".join(bench.collection[97]['text'])
chunks = chunk_text(text)
print(len(text))
print(len(chunks))

In [8]:
print(bench.collection[97]['title'])

Зло под солнцем


In [ ]:
s1 = await bench.blueprint.run(chunks, mode='cluster', initial_word_limit=500)

In [2]:
from utils import chunk_text
text = "\n".join(bench.collection[17]['text'])
chunks = chunk_text(text)
print(len(text))
for c in chunks:
    print(c)

16872
Иван Бунин Кастрюк Внезапно выскочив из - за крайней избы, с полевой дороги, во всю прыть маленьких лошадок летели по деревенской улице барчуки из Залесного. Подпрыгивая и хватаясь за холки, они гнались вперегонки, и ветер пузырями надувал на их спинах ситцевые рубашки. Теленок шарахнулся от них в сенцы, куры и впереди них петух, приседая к земле, неслись куда глаза глядят. Но отчаяннее всех улепетывала по деревенской улице маленькая белоголовая девочка в одной рубашонке. Обезумев от страха, она вскочила на огороды, несколько раз с размаху упала по дороге и вдруг увидала в воротах риги дедушку. С звонким криком бросилась она в его колени. - Что ты, что ты, дурочка? - закричал и дед, ловя ее за рубашку. - Барчуки... на жеребцах! - захлебываясь от слез, едва могла выговорить внучка. Дед усадил ее на колени, начал уговаривать. Внучка скоро затихла и, изредка всхлипывая, обиженным, вздрагивающим голосом начала рассказывать, как было дело. Поглаживая ее по голове, дед задумчиво улыбал

In [3]:
bench.collection[17]['title']

'Кастрюк'

In [4]:
s1 = await bench.hierarchical.run(chunks, filtered=False, initial_word_limit=500)

ONE CHUNK
Ниже приведена часть истории:
---
всходами и пестреющей паром, тонко, нежно засинел воздух. Безмятежнее и еще слаще, чем днем, заливались жаворонки. С паров пахло свежестью, зацветающими травами, медовой пылью желтого донника... Дед закрывал глаза, прислушивался, убаюкиваясь. " Эх, кабы теперь дождичка, - думал он, - то - то бы ржи - то поднялись! Да нет, опять солнышко чисто садится! " Вспоминая, что и завтра предстоит ему стариковский день, он морщился, придумывал, как бы избавиться от него. Он досадливо качал головою, скреб спину, облаченную в длинную стариковскую рубаху... и, наконец, пришел к счастливой мысли. - Ну, прикончил? - говорил он через полчаса заискивающим тоном, шагая рядом с сыном и держась за оглоблю сохи. - Кончить - то кончил, - отвечал Андрей ласково, - а ты - то как? Небось соскучился? - И - и, не приведи бог! - воскликнул дед ото всего сердца. - Сослужил, брат, службу... не хуже какого - нибудь солдата старого на капусте! И смеясь, не желая придавать св

In [165]:
import json

data_t = []
with open('qa_results/tpro.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data_t.append(json.loads(line))

In [166]:
methods = [
    "blueprint",
    "blueprint_cluster",
    "hierarchical",
    "hierarchical_filtered",
]

new_datat = []
for i in range(0, 400, 4):
    c = 0
    for m in methods:
        new_datat.append((data_t[i + c], m))
        c += 1

In [167]:
print(len(new_datat))

400


In [207]:
new_data = sorted(new_data, key=lambda x: x[0]['similarity'], reverse=True)

In [229]:
tmp = [e for e in new_data if e[1] == 'blueprint']

In [245]:
for i in new_data:
    if i[0]['title'] == 'Барбос и Жулька' and i[1] == 'blueprint':
        print(i)
        break

({'title': 'Барбос и Жулька', 'author': 'Куприн', 'questions': ['Какие два типа собак упоминаются в аннотации?  ', 'Кто прогнал бешеную собаку со двора?  ', 'Почему комнатная собачка заболела?  ', 'Что чувствует дворняга после болезни комнатной собачки?  ', 'Какова причина болезни комнатной собачки?  ', 'Кто является другом комнатной собачки?'], 'answers_gold': ['Дворняжка и комнатная собачка.', 'Комнатная собачка прогнала бешеную собаку со двора.', 'Комнатная собачка заболела, потому что прогнала забежавшую во двор бешеную собаку.', 'Дворняга чувствует смерть друга.', 'Причиной болезни комнатной собачки стало нападение бешеной собаки, которую она прогнала со двора.', 'Другом комнатной собачки является простая дворняжка.'], 'answers_gen': ['Барбос и Жулька.', 'Бешеную собаку со двора прогнала Жулька.', 'Комната́я соба́чка заболе́ла после́ того́, как броси́лась напере́рез бе́шеной со́баке, спаса́я двор, и в результа́те столкнове́ния с ней получи́ла боле́знь, от кото́рой слабе́ла, отказы

In [198]:
import numpy as np

rs_dict = { (i[0]['title'], i[1]) : i[0]['coverage'] for i in new_datat }
ds_dict = { (m[0]['title'], m[1]) : m[0]['coverage'] for m in new_data }

all_methods = set([k[1] for k in rs_dict.keys()]) | set([k[1] for k in ds_dict.keys()])

bins = np.arange(0, 1.1, 0.1)
bin_labels = [f"{bins[i]:.1f}–{bins[i+1]:.1f}" for i in range(len(bins)-1)]

for method in sorted(all_methods):
    diffs = []

    for key in rs_dict:
        if key[1] != method:
            continue
        if key in ds_dict:
            diff = ds_dict[key] - rs_dict[key]
            diffs.append(diff)

    diffs = np.array(diffs)
    
    if len(diffs) == 0:
        print(f"\nМетод: {method} — нет данных.")
        continue

    print(f"\nМетод: {method}")
    print("Count:", len(diffs))
    print("Mean:", round(np.mean(diffs), 6))
    print("Std:", round(np.std(diffs), 6))
    print("Min:", round(np.min(diffs), 6))
    print("25%:", round(np.percentile(diffs, 25), 6))
    print("50% (Median):", round(np.median(diffs), 6))
    print("75%:", round(np.percentile(diffs, 75), 6))
    print("Max:", round(np.max(diffs), 6))
    print("Negatives (diff < 0):", np.sum(diffs < 0))
    print("Above 0.1:", np.sum(diffs >= 0.1))

    bin_counts = np.histogram(diffs, bins=bins)[0]
    for label, count in zip(bin_labels, bin_counts):
        print(f"{label}: {count}")



Метод: blueprint
Count: 100
Mean: -0.006426
Std: 0.175345
Min: -0.647059
25%: -0.102778
50% (Median): 0.0
75%: 0.08
Max: 0.416667
Negatives (diff < 0): 43
Above 0.1: 22
0.0–0.1: 35
0.1–0.2: 9
0.2–0.3: 8
0.3–0.4: 4
0.4–0.5: 1
0.5–0.6: 0
0.6–0.7: 0
0.7–0.8: 0
0.8–0.9: 0
0.9–1.0: 0

Метод: blueprint_cluster
Count: 100
Mean: 0.086596
Std: 0.172331
Min: -0.3
25%: -0.005319
50% (Median): 0.062745
75%: 0.186364
Max: 0.65
Negatives (diff < 0): 25
Above 0.1: 38
0.0–0.1: 37
0.1–0.2: 13
0.2–0.3: 12
0.3–0.4: 8
0.4–0.5: 3
0.5–0.6: 1
0.6–0.7: 1
0.7–0.8: 0
0.8–0.9: 0
0.9–1.0: 0

Метод: hierarchical
Count: 100
Mean: 0.132976
Std: 0.162115
Min: -0.25
25%: 0.030556
50% (Median): 0.131884
75%: 0.236471
Max: 0.714286
Negatives (diff < 0): 16
Above 0.1: 58
0.0–0.1: 26
0.1–0.2: 29
0.2–0.3: 15
0.3–0.4: 8
0.4–0.5: 3
0.5–0.6: 2
0.6–0.7: 0
0.7–0.8: 1
0.8–0.9: 0
0.9–1.0: 0

Метод: hierarchical_filtered
Count: 100
Mean: 0.078679
Std: 0.150061
Min: -0.222222
25%: 0.0
50% (Median): 0.066667
75%: 0.15
Max: 0.571429

In [188]:
lst = [float(e/10) for e in range(10)]
diff = []
for l in range(9):
    diff = []
    for i in new_datar:
        if i[1] == 'blueprint':
            for m in new_data:
                if m[0]['title'] == i[0]['title'] and m[1] == i[1]:
                    if -m[0]['coverage'] + i[0]['coverage'] >= lst[l] and -m[0]['coverage'] + i[0]['coverage'] <= lst[l+1]:
                        diff.append(1)
    print(len(diff))
                

27
11
2
1
0
0
0
0
0


In [171]:
lst

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [244]:
for i in new_data:
    if i[0]['title'] == 'Барбос и Жулька' and i[1] == 'hierarchical':
        print(i)
        break

({'title': 'Барбос и Жулька', 'author': 'Куприн', 'questions': ['Какие два типа собак упоминаются в аннотации?  ', 'Кто прогнал бешеную собаку со двора?  ', 'Почему комнатная собачка заболела?  ', 'Что чувствует дворняга после болезни комнатной собачки?  ', 'Какова причина болезни комнатной собачки?  ', 'Кто является другом комнатной собачки?'], 'answers_gold': ['Дворняжка и комнатная собачка.', 'Комнатная собачка прогнала бешеную собаку со двора.', 'Комнатная собачка заболела, потому что прогнала забежавшую во двор бешеную собаку.', 'Дворняга чувствует смерть друга.', 'Причиной болезни комнатной собачки стало нападение бешеной собаки, которую она прогнала со двора.', 'Другом комнатной собачки является простая дворняжка.'], 'answers_gen': ['В аннотации упоминаются две собаки: Барбос — крупный, неуклюжий пёс с боевым нравом, и Жулька — маленькая, изящная собачка с чёрной шерстью и жёлтыми подпалинами, скромная и деликатная.', 'Бешеную собаку со двора прогнала Жулька.', 'Комнатная собачк

In [77]:
c = 0
for d in new_data:
    if d[1] == 'hierarchical_filtered' and d[0]['similarity'] < 0.2: 
        if c > 2:
            print(d)
            break
        c +=1

({'title': 'Встреча в Багдаде', 'author': 'Кристи', 'questions': ['Кто сообщил капитану Кросби о предстоящей встрече глав держав в Багдаде?  ', 'Кого ждали Кросби и Дейкин в Багдаде для получения ценной информации?  ', 'Почему шеф Анны Шейле удивился её просьбе об отпуске?  ', 'Кто предложил Виктории Джонс поехать в Багдад в качестве помощницы?  ', 'Как Виктория представилась миссис Клиппс, чтобы поехать в Багдад?  ', 'Кто спас Кармайкла от покушения в приёмной консула?  ', 'Кто привёз Ричарда Бейкера к себе домой после покушения на Кармайкла?  ', 'Что Ричард Бейкер обнаружил у себя в кармане после встречи с Кармайклом?  ', 'Кто из пассажиров летел в одном самолёте с Викторией и миссис Клиппс?  ', 'Что сэр Руперт Ли купил в лавке по пути в посольство в Багдаде?  ', 'Куда сэр Руперт Ли сообщил, что переезжает из посольства?  ', 'Где, по словам доктора Ратбоуна, находился Эдвард, когда Виктория приехала в Багдад?  ', 'Кто сообщил Виктории о существовании тайной организации, усиливающей п

In [57]:
print(len(new_data))

400


In [64]:
print(new_data[0][0]['questions'][7])

Какие изменения в окружающей среде замечают охотники?  


In [63]:
print(new_data[0][0]['answers_gen'][7])

In [65]:
print(new_data[0][0]['answers_gold'][7])

Охотники замечают, что цвета и химический состав атмосферы стали другими, изменились правила орфографии языка, а вместо президента-либерала у власти находится диктатор.


In [1]:
import time
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

#model_names = ['RefalMachine/RuadaptQwen2.5-7B-Lite-Beta', 'tpro', 'RefalMachine/RuadaptQwen3-32B-Instruct-v2', 'DeepSeek V3', 'yagpt5lite', 'Qwen3-235B-A22B-Instruct-2507']
model_names = ['tpro']
for name in model_names:
    print(name)
    bench.change_model(key, url, name)
    a1 = 0
    for _ in range(6):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'default')
        end = time.perf_counter()
        c = end - start
        print(c)
        a1 += c
    res = a1 / 6
    print(f'Blueprint: {res:.2f}')
    a2 = 0
    for _ in range(6):
        start = time.perf_counter()
        s = await bench.blueprint.run(chunks, 500, 'cluster')
        end = time.perf_counter()
        c = end - start
        print(c)
        a2 += c
    res = a2 / 6
    print(f'Blueprint cluster: {res:.2f}')
    a3 = 0
    for _ in range(6):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, False)
        end = time.perf_counter()
        c = end - start
        print(c)
        a3 += c
    res = a3 / 6
    print(f'Hierarchical: {res:.2f}')
    a4 = 0
    for _ in range(6):
        start = time.perf_counter()
        s = await bench.hierarchical.run(chunks, 500, True)
        end = time.perf_counter()
        c = end - start
        print(c)
        a4 += c
    res = a4 / 6
    print(f'Hierarchical filtered: {res:.2f}')
    print()

81049
chunks:  11
tpro
Кто такие Майк Энслин и мистер Олин?  
Каковы отношения между Майком и мистером Олином?  
Какие книги написал Майк и почему они важны для него?  
Как мистер Олин относится к книгам Майка?  
Какую роль играет минидиктофон в разговоре?  
Почему Майк хочет снять номер 1408?  
Какие законы Майк упоминает в поддержку своего требования?  
Какие чувства испытывает Майк, когда его книги читают другие?  
Что удивило мистера Олина в книгах Майка?  
Как Майк реагирует на слова мотивации мистера Олина?  
Что мистер Олин находит тревожным в книгах Майка?  
Как Майк относится к собственным историям о призраках?  
Какие примеры из книг Майка приводит мистер Олин?  
Почему Майк выключает минидиктофон?  
Какие ожидания Майка о мистере Олине не оправдались?
QUESTIONS
15
['Кто такие Майк Энслин и мистер Олин?', 'Каковы отношения между Майком и мистером Олином?', 'Какие книги написал Майк и почему они важны для него?', 'Как мистер Олин относится к книгам Майка?', 'Какую роль играет 

CancelledError: 

In [2]:
import time
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(URL=url, KEY=key, model_name='tpro', device=device, encoder=encoder)

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

start = time.perf_counter()
s = await bench.blueprint.run(chunks, 500, 'default')
end = time.perf_counter()
c = end - start
print(c)

81049
chunks:  11
QUESTIONS
15
['Кто такой Майк и какова его профессия?', 'Кто такой Олин и какова его роль в истории?', 'Какое значение имеет номер 1408 в отеле "Дельфин"?', 'Как Майк реагирует на перекос двери номера 1408?', 'Какую роль играет минидиктофон в истории Майка?', 'Какие странные явления Майк замечает в номере 1408?', 'Как Майк объясняет свои ощущения в номере 1408?', 'Какие отношения у Майка с Олином?', 'Какие предыдущие экспедиции Майка упоминаются в тексте?', 'Как Майк относится к паранормальным явлениям?', 'Какие инсинуации Олина вызывают у Майка раздражение?', 'Как Майк использует минидиктофон в своих экспедициях?', 'Какие изменения происходят с дверью номера 1408 при взгляде на неё?', 'Как Майк узнает о камере наблюдения в коридоре?', 'Как Майк входит в номер 1408 и что он замечает внутри?']
QUESTIONS
15
['Кто является главным персонажем текста?', 'Что такое книжица спичек и почему она важна для Майка?', 'Какие автомобили и униформа швейцара указывают на временной ко

In [3]:
import time
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)

text = "\n".join(bench.collection[78]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

start = time.perf_counter()
s = await bench.blueprint.run(chunks, 500, 'default')
end = time.perf_counter()
c = end - start
print(c)

81049
chunks:  11
QUESTIONS
15
['Кто спас Майка от серьезных ожогов и как?', 'Какие последствия ожогов остались у Майка?', 'Как ожоги спасли Майку жизнь?', 'Что произошло бы с Майком, если бы не спички с предупреждением?', 'Какие книги написал Майк до рокового события?', 'Почему Майк больше не может писать?', 'Какие физические и психические проблемы испытывает Майк после происшествия?', 'Как Майк описывает свои кошмары?', 'Что Майк понял о номере 1408 во время прогулки?', 'Почему Майк спит с включенным светом?', 'Почему он убрал все телефоны из дома?', 'Как Майк реагирует на закатный свет?', 'Какие изменения в своем состоянии ожидает Майк со временем?', 'Кто такой Олин и что он пытался сказать Майку?', 'Как Майк характеризует сущность, обитавшую в номере 1408?']
QUESTIONS
15
['1. Кто такой Майк Энслин и чем он занимается?', '2. Какие события из прошлого Майка упоминаются в тексте?', '3. Почему Майк не верит в призраков, несмотря на свои исследования?', '4. Кто такой Олин и какова его р

In [1]:
import time
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
import re
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)

bench1 = Summarisation(URL=url, KEY=key, model_name='tpro', device=device, encoder=encoder)
bench2 = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)
bench3 = Summarisation(URL=url, KEY=key, model_name='RefalMachine/RuadaptQwen3-32B-Instruct-v2', device=device, encoder=encoder)

for _ in range(3):
    print('tpro')
    start = time.perf_counter()
    s = await bench1.client.get_completion('Расскажи мне все об LLM!', max_tokens=1024, temperature=0.01)
    end = time.perf_counter()
    c = end - start
    print(c)

    print('deepseek')
    start = time.perf_counter()
    s = await bench2.client.get_completion('Расскажи мне все об LLM!', max_tokens=1024, temperature=0.001)
    end = time.perf_counter()
    c = end - start
    print(c)

    print('ruadapt')
    start = time.perf_counter()
    s = await bench3.client.get_completion('Расскажи мне все об LLM! no_think', max_tokens=1024, temperature=0.01)
    end = time.perf_counter()
    c = end - start
    print(c)

tpro
!!! 0.01
33.29884682409465
deepseek
!!! 0.001
17.101041697897017
ruadapt
!!! 0.01
45.14668290899135
tpro
!!! 0.01
31.10650725592859
deepseek
!!! 0.001
18.10435081506148
ruadapt
!!! 0.01
41.12261502305046
tpro
!!! 0.01
32.11626898101531
deepseek
!!! 0.001
17.104354810202494
ruadapt
!!! 0.01
45.132443451089785


In [2]:
import time
from utils import chunk_text
import sys
import torch
from sentence_transformers import SentenceTransformer
sys.path.append('methods')

from methods import Summarisation
with open('Access_key.txt', 'r', encoding='utf-8') as file:
    url, key = file.read().split()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = SentenceTransformer('deepvk/USER-bge-m3').to(device)
bench = Summarisation(URL=url, KEY=key, model_name='DeepSeek V3', device=device, encoder=encoder)

text = "\n".join(bench.collection[0]['text'])
chunks = chunk_text(text)
print(len(text))
print('chunks: ', len(chunks))

start = time.perf_counter()
s = await bench.blueprint.run(chunks, 500, 'default')
end = time.perf_counter()
c = end - start
print(c)

60664
chunks:  8
!!! 0.01
!!! 0.01
!!! 0.01
!!! 0.01
!!! 0.01
!!! 0.01
!!! 0.01
!!! 0.01
Кто остановил Мейзи и как она отреагировала на это?  
Какие эмоции и действия вызвало появление незнакомой руки у духов?  
Кто такие Гедда и Иоланта и как они попрощались с Мейзи?  
Как вел себя студент, остановивший Мейзи, и что он ей предложил?  
Как Мейзи оказалась в постели и что она почувствовала, придя в себя?  
Что случилось с башней после того, как Мейзи спустили вниз?  
Как доктор и миссис Уэст объяснили произошедшее с Мейзи?  
Как Мейзи отреагировала на новость о разрушении башни?  
Что стало со старухой Бесси и как Мейзи восприняла эту новость?  
Кого Мейзи назвала "двумя милыми девушками" и что ответила миссис Уэст?
QUESTIONS
10
['Кто остановил Мейзи и как она отреагировала на это?', 'Какие эмоции и действия вызвало появление незнакомой руки у духов?', 'Кто такие Гедда и Иоланта и как они попрощались с Мейзи?', 'Как вел себя студент, остановивший Мейзи, и что он ей предложил?', 'Как Мей